# Sistema de Enriquecimiento Filatelico

In [ ]:
# ============================================================================
# IMPORTS Y CONFIGURACIÓN - SISTEMA BILINGÜE
# ============================================================================

import sys
import os
from pathlib import Path
from typing import Dict, List, Tuple, Optional, Any
import glob
from pathlib import Path
from typing import List, Dict, Optional, Tuple
from datetime import datetime
import copy

# Import our new bilingual philatelic logic module
from philatelic_chunk_logic import *

# Initialize main enricher
global_enricher = SemanticEnricher()

## Combine Chunks - Mejora de Contexto Semántico

Lógica de backtracking para combinar chunks problemáticos donde párrafos cortos (`para`) pierden contexto al estar separados de sus headers/secciones (`sec`) correspondientes.

## Objetivo
- Mejorar la calidad de los chunks para indexación en Weaviate
- Mantener contexto semántico combinando headers con párrafos relacionados
- Procesar todos los PDFs de `results/parsed_jsons/` y generar versiones mejoradas en `results/final_jsons/`

## 1. Setup

In [ ]:
# Configuración de directorios
BASE_DIR = Path('.')
PARSED_JSONS_DIR = BASE_DIR / 'results' / 'parsed_jsons'
MARKDOWN_DIR = BASE_DIR / 'results' / 'markdown'
FINAL_JSONS_DIR = BASE_DIR / 'results' / 'final_jsons'

# Crear directorio de salida si no existe
FINAL_JSONS_DIR.mkdir(parents=True, exist_ok=True)

print(f"Directorios configurados:")
print(f"- Input JSON: {PARSED_JSONS_DIR}")
print(f"- Input Markdown: {MARKDOWN_DIR}")
print(f"- Output Final: {FINAL_JSONS_DIR}")

## 2. Función Principal de Combinación

In [ ]:
def combine_chunks(sec_chunk: Dict, para_chunk: Dict) -> Dict:
    """
    Combina un chunk de sección (sec) con un chunk de párrafo (para).
    
    El chunk resultante tendrá:
    - Texto: sec_text + "\n\n" + para_text
    - Metadata combinada inteligentemente
    - Grounding de ambos chunks
    """
    # Crear una copia profunda del chunk de sección como base
    combined_chunk = copy.deepcopy(sec_chunk)
    
    # Combinar textos
    sec_text = sec_chunk.get('text', '').strip()
    para_text = para_chunk.get('text', '').strip()
    
    combined_text = f"{sec_text}\n\n{para_text}" if sec_text and para_text else sec_text + para_text
    combined_chunk['text'] = combined_text
    
    # Actualizar chunk_id para reflejar la combinación
    sec_id = sec_chunk.get('chunk_id', '')
    para_id = para_chunk.get('chunk_id', '')
    combined_chunk['chunk_id'] = f"{sec_id}+{para_id.split(':')[-1]}"  # Formato más limpio
    
    # Combinar grounding (coordenadas de ubicación)
    sec_grounding = sec_chunk.get('grounding', [])
    para_grounding = para_chunk.get('grounding', [])
    combined_chunk['grounding'] = sec_grounding + para_grounding
    
    # Combinar metadata
    if 'metadata' in combined_chunk and 'metadata' in para_chunk:
        # Combinar labels manteniendo 'sec' como principal
        sec_labels = set(combined_chunk['metadata'].get('labels', []))
        para_labels = set(para_chunk['metadata'].get('labels', []))
        
        # El chunk combinado mantiene 'sec' pero añade información de que incluye 'para'
        combined_labels = list(sec_labels | para_labels)
        combined_chunk['metadata']['labels'] = combined_labels
        
        # Combinar reading_order_range
        sec_range = combined_chunk['metadata'].get('reading_order_range', [0, 0])
        para_range = para_chunk['metadata'].get('reading_order_range', [0, 0])
        
        combined_range = [
            min(sec_range[0], para_range[0]),
            max(sec_range[1], para_range[1])
        ]
        combined_chunk['metadata']['reading_order_range'] = combined_range
        
        # Actualizar combined_chunks counter
        combined_chunk['metadata']['combined_chunks'] = (
            combined_chunk['metadata'].get('combined_chunks', 1) + 
            para_chunk['metadata'].get('combined_chunks', 1)
        )
        
        # Promedio de quality_score si existe
        sec_quality = combined_chunk['metadata'].get('quality_score', 0.5)
        para_quality = para_chunk['metadata'].get('quality_score', 0.5)
        combined_chunk['metadata']['quality_score'] = (sec_quality + para_quality) / 2
    
    return combined_chunk

## 3. Procesamiento de Documentos

In [ ]:
def process_document_chunks(doc_data: Dict) -> Tuple[Dict, Dict]:
    """
    Procesa los chunks aplicando la LÓGICA CORREGIDA DE REUTILIZACIÓN con ENRIQUECIMIENTO SEMÁNTICO:
    - TODOS los chunks de texto buscan su header anterior
    - Headers pueden ser REUTILIZADOS para múltiples chunks
    - Cada chunk simplemente busca el header más cercano disponible
    - NO hay concepto de "headers usados" - todos son reutilizables
    - NUEVO: Integra enriquecimiento semántico usando philatelic_chunk_logic.py
    
    Returns:
        Tuple[Dict, Dict]: (processed_document, stats)
    """
    chunks = doc_data.get('chunks', [])
    processed_chunks = []
    
    # Inicializar ChunkCombiner con el enricher global
    from philatelic_chunk_logic import ChunkCombiner
    combiner = ChunkCombiner(global_enricher)
    
    stats = {
        'original_chunks': len(chunks),
        'combined_chunks': 0,
        'final_chunks': 0,
        'combinations': [],
        'no_header_found': 0,
        'estimated_size_bytes': 0,
        'largest_chunk_size': 0,
        'header_reuse_stats': {},  # Estadísticas de reutilización de headers
        'unique_headers_used': set(),  # Para tracking informativo
        'semantic_enrichment_applied': 0  # NUEVO: contador de enriquecimientos semánticos
    }
    
    print(f"📄 Procesando {len(chunks)} chunks con lógica de REUTILIZACIÓN de headers + ENRIQUECIMIENTO SEMÁNTICO...")
    
    for i, current_chunk in enumerate(chunks):
        try:
            # Verificar estructura del chunk
            if not isinstance(current_chunk, dict):
                print(f"    ⚠️ Chunk {i} inválido: {type(current_chunk)}")
                processed_chunks.append(current_chunk)
                continue
            
            current_labels = get_chunk_labels(current_chunk)
            current_text = current_chunk.get('text', '')
            
            # NUEVA LÓGICA CON CLASES INTEGRADAS: Usar ChunkCombiner
            should_combine, header_index = combiner.should_combine_chunks(current_chunk, processed_chunks)
            
            if should_combine and header_index is not None:
                # Combinar con el header encontrado usando enriquecimiento semántico
                header_chunk = processed_chunks[header_index]
                
                try:
                    # NUEVO: Usar ChunkCombiner que integra enriquecimiento semántico automáticamente
                    combined = combiner.combine_single_header_chunk(header_chunk, current_chunk)
                    
                    # Verificar tamaño del resultado
                    chunk_size = estimate_chunk_size_bytes(combined)
                    stats['largest_chunk_size'] = max(stats['largest_chunk_size'], chunk_size)
                    
                    # Límite de seguridad: 100KB por chunk
                    MAX_CHUNK_SIZE = 100 * 1024  # 100KB
                    if chunk_size > MAX_CHUNK_SIZE:
                        print(f"    ⚠️ Chunk {i}: Combinación muy grande ({chunk_size:,} bytes), usando original")
                        processed_chunks.append(current_chunk)
                        continue
                    
                    stats['estimated_size_bytes'] += chunk_size
                    
                    # Verificar si se aplicó enriquecimiento semántico
                    if combined.get('metadata', {}).get('semantic_enrichment_applied', False):
                        stats['semantic_enrichment_applied'] += 1
                    
                except Exception as combine_error:
                    print(f"    ❌ Error combinando chunk {i}: {str(combine_error)[:50]}...")
                    processed_chunks.append(current_chunk)
                    continue
                
                # ÉXITO: Registrar estadísticas (pero NO marcar header como "usado")
                stats['combined_chunks'] += 1
                
                # Estadísticas de reutilización de headers
                header_id = header_chunk.get('chunk_id', 'unknown')
                header_labels = get_chunk_labels(header_chunk)
                header_text = header_chunk.get('text', '')[:30] + '...'
                
                # Contar cuántas veces se reutiliza cada header
                if header_id not in stats['header_reuse_stats']:
                    stats['header_reuse_stats'][header_id] = {
                        'labels': header_labels,
                        'text_preview': header_text,
                        'reuse_count': 0,
                        'combined_with': []
                    }
                
                stats['header_reuse_stats'][header_id]['reuse_count'] += 1
                stats['header_reuse_stats'][header_id]['combined_with'].append({
                    'chunk_id': current_chunk.get('chunk_id', 'unknown'),
                    'labels': current_labels
                })
                
                # Tracking único de headers (para estadísticas)
                stats['unique_headers_used'].add(header_id)
                
                # Registrar ejemplo de combinación (simplificado)
                stats['combinations'].append({
                    'content_id': current_chunk.get('chunk_id', 'unknown')[:15],
                    'content_labels': ', '.join(current_labels),
                    'header_labels': ', '.join(header_labels),
                    'header_preview': header_text,
                    'combined_size_kb': round(chunk_size / 1024, 1),
                    'header_reuse_count': stats['header_reuse_stats'][header_id]['reuse_count'],
                    'semantic_enriched': combined.get('metadata', {}).get('semantic_enrichment_applied', False)  # NUEVO
                })
                
                # Añadir chunk combinado
                processed_chunks.append(combined)
                
                enrichment_status = "🎯" if combined.get('metadata', {}).get('semantic_enrichment_applied', False) else "📝"
                print(f"    ✅ Chunk {i} ({', '.join(current_labels)}): combinado con {', '.join(header_labels)} (reutilización #{stats['header_reuse_stats'][header_id]['reuse_count']}) {enrichment_status}")
                
            else:
                # No se pudo combinar - aplicar enriquecimiento semántico individual
                if any(label in ['para', 'marginalia', 'tab'] for label in current_labels):
                    try:
                        # NUEVO: Aplicar enriquecimiento semántico aún sin header
                        enriched_chunk = global_enricher.enrich_chunk_advanced_bilingual(current_chunk.copy())
                        if enriched_chunk.get('metadata', {}).get('quality_score', 0) > current_chunk.get('metadata', {}).get('quality_score', 0):
                            current_chunk = enriched_chunk
                            stats['semantic_enrichment_applied'] += 1
                            print(f"    🎯 Chunk {i} ({', '.join(current_labels)}): enriquecimiento semántico individual aplicado")
                    except Exception as enrich_error:
                        print(f"    ⚠️ Error enriqueciendo chunk {i}: {str(enrich_error)[:30]}...")
                    
                    stats['no_header_found'] += 1
                    print(f"    📝 Chunk {i} ({', '.join(current_labels)}): sin header disponible")
                
                # Añadir chunk sin combinar (pero posiblemente enriquecido)
                chunk_size = estimate_chunk_size_bytes(current_chunk)
                stats['estimated_size_bytes'] += chunk_size
                stats['largest_chunk_size'] = max(stats['largest_chunk_size'], chunk_size)
                processed_chunks.append(current_chunk)
                
        except Exception as e:
            # Error general
            error_info = f"{type(e).__name__}: {str(e)[:50]}..."
            print(f"    ❌ Error en chunk {i}: {error_info}")
            
            processed_chunks.append(current_chunk)
    
    stats['final_chunks'] = len(processed_chunks)
    
    # Verificar eficiencia del algoritmo
    size_mb = stats['estimated_size_bytes'] / (1024 * 1024)
    combination_rate = (stats['combined_chunks'] / stats['original_chunks']) * 100 if stats['original_chunks'] > 0 else 0
    enrichment_rate = (stats['semantic_enrichment_applied'] / stats['original_chunks']) * 100 if stats['original_chunks'] > 0 else 0
    
    # Estadísticas de reutilización
    total_reuses = sum(header_stats['reuse_count'] for header_stats in stats['header_reuse_stats'].values())
    unique_headers_count = len(stats['unique_headers_used'])
    avg_reuse_per_header = total_reuses / unique_headers_count if unique_headers_count > 0 else 0
    
    print(f"\n📊 RESULTADOS:")
    print(f"  Chunks combinados: {stats['combined_chunks']}/{stats['original_chunks']} ({combination_rate:.1f}%)")
    print(f"  Enriquecimiento semántico: {stats['semantic_enrichment_applied']}/{stats['original_chunks']} ({enrichment_rate:.1f}%)")  # NUEVO
    print(f"  Headers únicos utilizados: {unique_headers_count}")
    print(f"  Promedio reutilización por header: {avg_reuse_per_header:.1f}")
    print(f"  Sin header encontrado: {stats['no_header_found']}")
    print(f"  Tamaño estimado: {size_mb:.2f} MB")
    print(f"  Chunk más grande: {stats['largest_chunk_size']//1024}KB")
    
    # Mostrar headers más reutilizados (top 3)
    most_reused = sorted(stats['header_reuse_stats'].items(), 
                        key=lambda x: x[1]['reuse_count'], reverse=True)[:3]
    
    if most_reused:
        print(f"  Headers más reutilizados:")
        for header_id, header_info in most_reused:
            reuse_count = header_info['reuse_count']
            labels = ', '.join(header_info['labels'])
            preview = header_info['text_preview']
            print(f"    - {labels}: \"{preview}\" (usado {reuse_count} veces)")
    
    # Crear documento procesado
    processed_doc = copy.deepcopy(doc_data)
    processed_doc['chunks'] = processed_chunks
    
    # Metadata del documento
    if 'metadata' not in processed_doc:
        processed_doc['metadata'] = {}
    
    processed_doc['metadata']['processing_date'] = datetime.now().isoformat()
    processed_doc['metadata']['header_reuse_strategy_applied'] = True
    processed_doc['metadata']['semantic_enrichment_applied'] = True  # NUEVO
    processed_doc['metadata']['original_chunk_count'] = stats['original_chunks']
    processed_doc['metadata']['final_chunk_count'] = stats['final_chunks']
    processed_doc['metadata']['combined_chunk_count'] = stats['combined_chunks']
    processed_doc['metadata']['semantic_enriched_count'] = stats['semantic_enrichment_applied']  # NUEVO
    processed_doc['metadata']['combination_rate_percent'] = round(combination_rate, 1)
    processed_doc['metadata']['enrichment_rate_percent'] = round(enrichment_rate, 1)  # NUEVO
    processed_doc['metadata']['unique_headers_used'] = unique_headers_count
    processed_doc['metadata']['avg_header_reuse'] = round(avg_reuse_per_header, 1)
    processed_doc['metadata']['estimated_size_mb'] = round(size_mb, 2)
    
    return processed_doc, stats

In [ ]:
def process_all_documents() -> Dict[str, Any]:
    """
    Procesa todos los archivos *_philatelic.json aplicando header backtracking + enriquecimiento semántico.
    
    Returns:
        Dictionary con estadísticas completas del procesamiento
    """
    # Obtener todos los archivos philatelic usando la función del módulo
    philatelic_files = get_all_philatelic_files()
    
    if not philatelic_files:
        print("❌ No se encontraron archivos *_philatelic.json en results/parsed_jsons/")
        return {
            'total_files': 0,
            'processed_files': 0,
            'failed_files': 0,
            'errors': [],
            'total_original_chunks': 0,
            'total_final_chunks': 0,
            'total_combinations': 0,
            'total_semantic_enrichments': 0,
            'total_size_mb': 0.0
        }
    
    print(f"📁 Encontrados {len(philatelic_files)} archivos para procesar")
    
    # Estadísticas globales
    global_stats = {
        'total_files': len(philatelic_files),
        'processed_files': 0,
        'failed_files': 0,
        'errors': [],
        'total_original_chunks': 0,
        'total_final_chunks': 0,
        'total_combinations': 0,
        'total_semantic_enrichments': 0,
        'total_size_mb': 0.0,
        'file_details': [],
        'size_warnings': [],
        'efficiency_report': {}
    }
    
    for i, file_path in enumerate(philatelic_files):
        try:
            doc_id = extract_doc_id_from_filename(file_path.name)
            print(f"\n[{i+1}/{len(philatelic_files)}] Procesando {doc_id}...")
            
            # Cargar archivo JSON
            with open(file_path, 'r', encoding='utf-8') as f:
                doc_data = json.load(f)
            
            original_chunks_count = len(doc_data.get('chunks', []))
            
            # Procesar el documento
            processed_doc, file_stats = process_document_chunks(doc_data)
            
            # Generar nombre de archivo de salida
            output_filename = f"{doc_id}_final.json"
            output_path = FINAL_JSONS_DIR / output_filename
            
            # Verificar tamaño antes de guardar
            estimated_size_mb = file_stats.get('estimated_size_mb', 0)
            if estimated_size_mb > 50:  # Warning para archivos grandes
                global_stats['size_warnings'].append({
                    'file': doc_id,
                    'final_mb': estimated_size_mb,
                    'reason': 'Large file size'
                })
            
            # Guardar archivo procesado
            with open(output_path, 'w', encoding='utf-8') as f:
                json.dump(processed_doc, f, ensure_ascii=False, indent=2)
            
            # Actualizar estadísticas globales
            global_stats['processed_files'] += 1
            global_stats['total_original_chunks'] += file_stats['original_chunks']
            global_stats['total_final_chunks'] += file_stats['final_chunks']
            global_stats['total_combinations'] += file_stats['combined_chunks']
            global_stats['total_semantic_enrichments'] += file_stats.get('semantic_enrichment_applied', 0)
            global_stats['total_size_mb'] += estimated_size_mb
            
            # Detalles por archivo
            global_stats['file_details'].append({
                'doc_id': doc_id,
                'original_chunks': file_stats['original_chunks'],
                'final_chunks': file_stats['final_chunks'],
                'combinations': file_stats['combined_chunks'],
                'semantic_enrichments': file_stats.get('semantic_enrichment_applied', 0),
                'file_size_mb': estimated_size_mb
            })
            
            print(f"✅ {doc_id}: {file_stats['combined_chunks']} combinaciones + {file_stats.get('semantic_enrichment_applied', 0)} enriquecimientos → {output_filename}")
            
        except Exception as e:
            error_msg = str(e)
            global_stats['failed_files'] += 1
            global_stats['errors'].append({
                'file': file_path.name,
                'error': error_msg
            })
            
            print(f"❌ Error procesando {file_path.name}: {error_msg[:60]}...")
            continue
    
    # Generar reporte de eficiencia
    if global_stats['processed_files'] > 0:
        avg_combinations = global_stats['total_combinations'] / global_stats['processed_files']
        avg_enrichments = global_stats['total_semantic_enrichments'] / global_stats['processed_files']
        avg_size = global_stats['total_size_mb'] / global_stats['processed_files']
        
        global_stats['efficiency_report'] = {
            'avg_combinations_per_file': round(avg_combinations, 1),
            'avg_enrichments_per_file': round(avg_enrichments, 1),
            'avg_file_size_mb': round(avg_size, 1),
            'memory_efficient_files': sum(1 for details in global_stats['file_details'] 
                                        if details['file_size_mb'] < 10),
            'combination_success_rate': round((global_stats['total_combinations'] / 
                                             max(global_stats['total_original_chunks'], 1)) * 100, 1),
            'enrichment_success_rate': round((global_stats['total_semantic_enrichments'] / 
                                            max(global_stats['total_original_chunks'], 1)) * 100, 1)
        }
    
    return global_stats

## 4. TEST DE CHUNK REAL CON ENRIQUECIMIENTO COMPLETO

In [ ]:
print("🚀 TEST CHUNK REAL - ENRIQUECIMIENTO SEMÁNTICO COMPLETO")
print("=" * 80)

# Chunk real de tabla filatélica para testing
test_table_chunk = {
    "chunk_id": "Oxcart253:025:3-3:0",
    "chunk_type": "text",
    "text": "0\t1\t2\r\nScott 147\tGR20 Type G7 2c used perf 12x11.5 wove paper lithographed mint never hinged very fine\t25.00\r\nMichel 23a\tGR21-28 Type G13 mostly used (ex-Saenz collection) comb perf yellow gum\t40.00\r\nYvert 45\tGR29-36 Type G14 postally used (ex-Saenz collection) thin spot crease\t40.00\r\n66\tGR38-45 Type G8 mint lightly hinged (ex-Colonel Green collection) superb centering\t20.00\r\n67\tGR47 Type G9 block of 4 mint no gum imperforate (ex-Alvarez) 1885 coat of arms\t5.00\r\n68\tG47-54 Type G9 not used (ex-Colonel Green collection) watermark inverted\t25.00\r\n70\tGR pair position 24 & 25 mint hinged gum signed Peralta 1880s cathedral design\t75.00\r\n71\tGR pair position 27 & 28 mint never hinged original gum signed Peralta engraved\t75.00\r\n72\tG3-4 pair not used gum signed Peralta and Napier laid paper extremely fine $50 USD\t75.00\r\n80\tRevenue Guanacaste Overprints Collection lots 48 to 80 inverted overprint color error\t869.-\r\n",
    "grounding": [{"page": 25, "box": None}],
    "metadata": {
        "labels": ["tab"],
        "reading_order_range": [3, 3]
    }
}

# Header simulado (como sería en el procesamiento real)
test_header_chunk = {
    "chunk_id": "Oxcart253:025:2-2:0",
    "chunk_type": "text", 
    "text": "COSTA RICA PHILATELIC AUCTION CATALOG 1885-1891 GUANACASTE OVERPRINT PERIOD",
    "metadata": {
        "labels": ["sec"],
        "reading_order_range": [2, 2]
    }
}

print("📋 CHUNK ORIGINAL (solo tabla):")
print(f"Texto: {test_table_chunk['text'][:100]}...")
print(f"Longitud: {len(test_table_chunk['text'])} caracteres")

print("\n" + "=" * 80)
print("🔍 APLICANDO ENRIQUECIMIENTO SEMÁNTICO...")

# 1. ENRIQUECIMIENTO INDIVIDUAL del chunk tabla
enriched_chunk = global_enricher.enrich_chunk_advanced_bilingual(test_table_chunk.copy())
entities = enriched_chunk.get('metadata', {}).get('entities', {})

print(f"\n📊 ENTIDADES EXTRAÍDAS:")
print(f"Quality Score: {enriched_chunk.get('metadata', {}).get('quality_score', 0):.2f}")

# Mostrar solo las entidades importantes encontradas
if entities.get('catalog'):
    sistemas = ', '.join(f"{c['system']} {c['number']}" for c in entities['catalog'][:3])
    print(f"Catálogos: {len(entities['catalog'])}, sistemas → {sistemas}")
if entities.get('condition'):
    print(f"Condiciones: {entities['condition']}")
if entities.get('perforation'):
    print(f"Perforación: {entities['perforation']}")
if entities.get('varieties'):
    print(f"EFO: {len(entities['varieties'])} variedades → {[v['label'] for v in entities['varieties']]}")
if entities.get('prices'):
    valores = ', '.join(f"{p['amount']} {p['currency']}" for p in entities['prices'][:3])
    print(f"Precios: {len(entities['prices'])} valores → {valores}")

print("\n" + "=" * 80)
print("🎯 COMBINACIÓN CON HEADER + ENRIQUECIMIENTO AVANZADO")

# 2. COMBINACIÓN CON HEADER usando ChunkCombiner (incluye enriquecimiento automático)
combiner = ChunkCombiner(global_enricher)
combined_chunk = combiner.combine_single_header_chunk(test_header_chunk, test_table_chunk)

print(f"\n📝 TEXTO ORIGINAL (Header + Tabla):")
original_combined = f"{test_header_chunk['text']}\\n\\n{test_table_chunk['text']}"
print(f"Longitud: {len(original_combined)} caracteres")

print(f"\n🚀 TEXTO FINAL ENRIQUECIDO:")
final_text = combined_chunk.get('text', '')
print(f"Longitud: {len(final_text)} caracteres")
print(f"Incremento: {((len(final_text) - len(original_combined)) / len(original_combined) * 100):.0f}%")

# Mostrar el resultado final de manera legible
print(f"\n📖 RESULTADO FINAL:")
print("-" * 40)
# Solo mostrar primeras líneas del texto enriquecido para ver la estructura
lines = final_text.split('\\n')
for i, line in enumerate(lines[:10]):  # Primeras 10 líneas
    if line.strip():
        print(f"{line.strip()}")

if len(lines) > 10:
    print(f"... [{len(lines)-10} líneas adicionales de enriquecimiento]")

print("-" * 40)

print(f"\n🎉 RESUMEN:")
print(f"✅ Chunk original: {len(test_table_chunk['text'])} chars")
print(f"✅ Con header: {len(original_combined)} chars") 
print(f"✅ Completamente enriquecido: {len(final_text)} chars")
print(f"✅ Mejora total: {((len(final_text) - len(test_table_chunk['text'])) / len(test_table_chunk['text']) * 100):.0f}%")
print(f"✅ Semantic enrichment aplicado: {combined_chunk.get('metadata', {}).get('semantic_enrichment_applied', False)}")

# Verificar qué tipos de enriquecimiento se aplicaron
enrichment_applied = []
if "Catálogos internacionales:" in final_text:
    enrichment_applied.append("Catálogos")
if "Especificaciones técnicas:" in final_text:
    enrichment_applied.append("Técnicas")
if "Condición:" in final_text:
    enrichment_applied.append("Condición")
if "Precios:" in final_text:
    enrichment_applied.append("Precios")
if "Guanacaste" in final_text:
    enrichment_applied.append("Contexto CR")

print(f"✅ Tipos de enriquecimiento aplicados: {', '.join(enrichment_applied) if enrichment_applied else 'Básico'}")

print("\\n🎯 ¡Listo para RAG! El chunk ahora tiene contexto completo y metadata rica.")

# 5. Procesamiento Masivo Todos los PDFs


In [ ]:
print("=" * 80)
print("🚀 INICIANDO PROCESAMIENTO MASIVO - NUEVA ESTRATEGIA PDF + ENRIQUECIMIENTO SEMÁNTICO")
print("TODOS LOS TEXTOS BUSCARÁN SU HEADER/SEC/SUB_SEC ANTERIOR + PHILATELIC ENRICHMENT")
print("=" * 80)

start_time = datetime.now()

# Ejecutar procesamiento con manejo de errores robusto
try:
    results = process_all_documents()
    processing_successful = True
except Exception as e:
    print(f"\n❌ ERROR DURANTE EL PROCESAMIENTO:")
    print(f"Tipo: {type(e).__name__}")
    print(f"Mensaje: {str(e)}")
    processing_successful = False
    results = {
        'total_files': 0,
        'processed_files': 0,
        'failed_files': 1,
        'errors': [{'file': 'procesamiento_general', 'error': str(e)}],
        'total_semantic_enrichments': 0
    }

end_time = datetime.now()
processing_time = (end_time - start_time).total_seconds()

print("\n" + "=" * 80)
print("📊 RESUMEN FINAL DEL PROCESAMIENTO")
print("=" * 80)

if processing_successful:
    print(f"✅ ÉXITO: Procesamiento completado con nueva estrategia PDF + ENRIQUECIMIENTO SEMÁNTICO")
    
    # Estadísticas principales
    print(f"\n📁 ARCHIVOS:")
    print(f"  Total encontrados:     {results.get('total_files', 0)}")
    print(f"  Procesados con éxito:  {results.get('processed_files', 0)}")
    print(f"  Fallidos:              {results.get('failed_files', 0)}")
    
    print(f"\n📊 CHUNKS:")
    print(f"  Chunks originales:     {results.get('total_original_chunks', 0):,}")
    print(f"  Chunks finales:        {results.get('total_final_chunks', 0):,}")
    print(f"  Combinaciones:         {results.get('total_combinations', 0):,}")
    print(f"  Enriquecimientos:      {results.get('total_semantic_enrichments', 0):,}")  # NUEVO
    
    # Calcular estadísticas
    if results.get('total_original_chunks', 0) > 0:
        combination_rate = (results.get('total_combinations', 0) / results.get('total_original_chunks', 0)) * 100
        enrichment_rate = (results.get('total_semantic_enrichments', 0) / results.get('total_original_chunks', 0)) * 100  # NUEVO
        print(f"  Tasa combinación:      {combination_rate:.1f}%")
        print(f"  Tasa enriquecimiento:  {enrichment_rate:.1f}%")  # NUEVO
    
    print(f"\n💾 TAMAÑOS:")
    print(f"  Tamaño total:          {results.get('total_size_mb', 0):.1f} MB")
    
    if results.get('processed_files', 0) > 0:
        avg_size = results.get('total_size_mb', 0) / results.get('processed_files', 0)
        print(f"  Tamaño promedio:       {avg_size:.1f} MB por archivo")
    
    print(f"\n⏱️ RENDIMIENTO:")
    print(f"  Tiempo total:          {processing_time:.1f} segundos")
    
    if results.get('processed_files', 0) > 0:
        avg_time = processing_time / results.get('processed_files', 0)
        print(f"  Tiempo promedio:       {avg_time:.1f}s por archivo")
    
    # Reportar eficiencia con enriquecimiento semántico
    efficiency_report = results.get('efficiency_report', {})
    if efficiency_report:
        print(f"\n📈 EFICIENCIA:")
        print(f"  Archivos eficientes:     {efficiency_report.get('memory_efficient_files', 0)}")
        print(f"  Combinaciones/archivo:   {efficiency_report.get('avg_combinations_per_file', 0)}")
        print(f"  Enriquecimientos/archivo: {efficiency_report.get('avg_enrichments_per_file', 0)}")  # NUEVO
        print(f"  Tasa éxito combinación:  {efficiency_report.get('combination_success_rate', 0):.1f}%")
        print(f"  Tasa éxito enriquecimiento: {efficiency_report.get('enrichment_success_rate', 0):.1f}%")  # NUEVO
    
    # Mostrar archivos más exitosos (combinando combinaciones + enriquecimientos)
    file_details = results.get('file_details', [])
    if file_details:
        print(f"\n🏆 TOP 5 ARCHIVOS CON MAYOR MEJORA TOTAL:")
        # Ordenar por la suma de combinaciones + enriquecimientos
        top_files = sorted(file_details, 
                          key=lambda x: x.get('combinations', 0) + x.get('semantic_enrichments', 0), 
                          reverse=True)[:5]
        
        for i, file_info in enumerate(top_files, 1):
            combinations = file_info.get('combinations', 0)
            enrichments = file_info.get('semantic_enrichments', 0)
            total_chunks = file_info.get('original_chunks', 0)
            file_size = file_info.get('file_size_mb', 0)
            total_improvements = combinations + enrichments
            
            if total_chunks > 0:
                improvement_rate = (total_improvements / total_chunks) * 100
                print(f"  {i}. {file_info.get('doc_id', 'unknown'):10} - {combinations} comb + {enrichments} enr = {total_improvements} mejoras ({improvement_rate:4.1f}%) - {file_size:.1f}MB")
    
    # Advertencias y errores
    size_warnings = results.get('size_warnings', [])
    if size_warnings:
        print(f"\n⚠️ ADVERTENCIAS DE TAMAÑO ({len(size_warnings)}):")
        for warning in size_warnings[:3]:  # Solo primeras 3
            print(f"  - {warning.get('file', 'unknown')}: {warning.get('final_mb', 0):.1f}MB ({warning.get('reason', 'unknown')})")
    
    errors = results.get('errors', [])
    if errors:
        print(f"\n❌ ERRORES ({len(errors)}):")
        for error in errors[:3]:  # Solo primeros 3
            print(f"  - {error.get('file', 'unknown')}: {str(error.get('error', 'unknown'))[:60]}...")
    
    print(f"\n📂 ARCHIVOS DE SALIDA:")
    print(f"  Directorio: {FINAL_JSONS_DIR}")
    print(f"  Formato: OXCART##_final.json")
    print(f"  ✅ Listos para indexación en Weaviate")
    
    print(f"\n🎯 NUEVA ESTRATEGIA PDF + ENRIQUECIMIENTO SEMÁNTICO - RESULTADOS:")
    print(f"✅ Chunks como 'POSITION 2: There are two blue dots...'")
    print(f"✅ AHORA tienen su contexto de header/sección anterior")
    print(f"✅ PLUS: Enriquecimiento semántico filatélico avanzado aplicado")
    print(f"✅ Detección de catálogos internacionales (Scott, Michel, Yvert, etc.)")
    print(f"✅ Clasificación EFO (Errores, variedades, oddities)")
    print(f"✅ Especificaciones técnicas (perforación, papel, impresión)")
    print(f"✅ Evaluación de condición (mint/used, centering, defects)")
    print(f"✅ Contexto Costa Rica (período Guanacaste, historia)")
    print(f"✅ Contexto semántico completo para RAG queries")
    print(f"✅ Tamaños controlados (no más archivos enormes)")
    print(f"✅ Deduplicación inteligente aplicada")

else:
    print(f"❌ FALLO: El procesamiento no se pudo completar")
    print(f"⏱️ Tiempo antes del fallo: {processing_time:.1f} segundos")

print(f"\n" + "=" * 80)
if processing_successful and results.get('processed_files', 0) > 0:
    total_improvements = results.get('total_combinations', 0) + results.get('total_semantic_enrichments', 0)
    print(f"🎉 ¡PROCESAMIENTO COMPLETADO CON ÉXITO!")
    print(f"La nueva estrategia PDF + ENRIQUECIMIENTO SEMÁNTICO funcionó perfectamente.")
    print(f"{total_improvements:,} chunks totales mejorados ({results.get('total_combinations', 0):,} combinaciones + {results.get('total_semantic_enrichments', 0):,} enriquecimientos).")
    print(f"Los textos finales son significativamente más ricos para embeddings RAG.")
else:
    print(f"⚠️ Procesamiento completado con limitaciones.")
    print(f"Revisar errores mostrados arriba.")
print("=" * 80)